In [33]:
import nltk
from nltk.corpus import cmudict, wordnet
from nltk.tokenize import word_tokenize, wordpunct_tokenize
import pprint
import itertools
import copy

In [36]:
nltk.download("cmudict")
nltk.download("wordnet")

[nltk_data] Downloading package cmudict to /home/ds/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ds/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [34]:
pp = pprint.PrettyPrinter(indent=4)

VOWELS = ['A', 'E', 'I', 'O', 'U']
NUMBER_CHOICES = [0, 1]

input1 = "eye mull of mush sheen"
output1 = "i'm a love machine"

In [35]:
def tokenize_sentence(sentence):
    return sentence.split()


def get_syllables(w):
    return [pron for (word, pron) in cmudict.entries() if word == w]


def get_syllables_of_sentence(sentence):
    return [get_syllables(word) for word in sentence]

In [37]:
def possible_pronunciations(syllables):
    pronunciations = [t for t in itertools.product(*syllables)]
    outer = []
    for pronunciation in pronunciations:
        inner = []
        for word in pronunciation:
            inner += word
        outer += [inner]
    return outer

In [38]:
def post_processing(pronunciations):
    # turn AH0 -> [AH0, AH1, EH0, EH1, IH0, IH1, OH0, OH1, UH0, UH1]
    # two or more of a sound gets knocked off [SH, SH] -> [[SH, ''], [SH]]
    #                                           ->[[SH, SH], ['', SH]]
    # turn B -> [B]
    # then do cartesian product to build all possible strings
    copy_of_pronunciations = copy.deepcopy(pronunciations)
    for pronunciation in pronunciations:
        for sound_loc in range(len(pronunciation)):
            sound = pronunciation[sound_loc]
            if sound_loc + 1 < len(pronunciation) \
                    and pronunciation[sound_loc + 1] == pronunciation[sound_loc]:
                deal_with_duplicates(pronunciation, sound_loc)
            if pronunciation[sound_loc][0] in VOWELS:
                deal_with_vowels(pronunciation, sound_loc)
            if pronunciation[sound_loc] == sound:
                pronunciation[sound_loc] = [pronunciation[sound_loc]]
        copy_of_pronunciations += itertools.product(*pronunciation)
    return copy_of_pronunciations

In [39]:
def deal_with_duplicates(pronunciation, sound_loc):
    pronunciation[sound_loc] = [pronunciation[sound_loc], '']


def deal_with_vowels(pronunciation, sound_loc):
    middle_letter = pronunciation[sound_loc][1]
    pronunciation[sound_loc] = [vowel + middle_letter + str(number)
                                for vowel in VOWELS
                                for number in NUMBER_CHOICES]

In [40]:
def remove_empty_quotes(pronunciations):
    # exists to remove empty quotes in pronunciations
    # turn [[SH, SH], ['', SH]] -> [[SH, SH], [SH]]
    for pronunciation_loc in range(len(pronunciations)):
        pronunciation = pronunciations[pronunciation_loc]
        pronunciations[pronunciation_loc] = [sound
                                             for sound in pronunciation
                                             if sound != '']
    return pronunciations

In [41]:
def do_two_strings_match(input_pronunciations, output_pronunciations):
    return any(output_pronunciation == list(input_pronunciation)
               for input_pronunciation in input_pronunciations
               for output_pronunciation in output_pronunciations)

In [42]:
input1_syllables = get_syllables_of_sentence(tokenize_sentence(input1))
output1_syllables = get_syllables_of_sentence(tokenize_sentence(output1))

In [43]:
input_pronunciations = remove_empty_quotes(
        post_processing(
            possible_pronunciations(input1_syllables)))
prettied_input_pronunciations = set([tuple(x) for x in input_pronunciations])
output_pronunciations = possible_pronunciations(output1_syllables)

In [44]:
do_two_strings_match(prettied_input_pronunciations, output_pronunciations)

True